# LOG
授業のニューラルネットワークの改良版でやったけど精度出ないのでとりあえずtensorflowの花分類チュートリアルから中間層を拝借してみる。これは拝借前のバックアップ

In [71]:
# 機械学習のライブラリ関連をインポート
import pandas as pd
import numpy as np

# ランダムにシャッフルして，学習・テストに分割するモジュール
from sklearn.model_selection import ShuffleSplit

# 深層学習のライブラリをインポート
import tensorflow as tf
import tensorflow.keras as keras

#表示系のインポートと設定
%matplotlib inline
import matplotlib.pyplot as plt

# ファイル操作
import os

# 画像操作
from PIL import Image

In [401]:
# DIRS
DIRS_DATASET = "../Training/Datasets10000/"

In [402]:
file_names = []
    
# フォルダ内のファイルを取得
files = os.listdir(DIRS_DATASET)

# ファイル名を配列に格納
for file in files:
    file_names.append(file)

# Pythonリスト型をnumpy.ndarray型に変換
file_names = np.array(file_names)


In [403]:
file_names

array(['4510033397528.jpeg', '4510050634699.jpeg', '4510072948944.jpeg',
       '4510077351076.jpeg', '4510104237612.jpeg', '4510116129826.jpeg',
       '4510125950222.jpeg', '4510135916152.jpeg', '4510148101379.jpeg',
       '4510154837844.jpeg', '4510155462178.jpeg', '4510189389132.jpeg',
       '4510192340546.jpeg', '4510245148364.jpeg', '4510250168878.jpeg',
       '4510283724299.jpeg', '4510299152040.jpeg', '4510311653255.jpeg',
       '4510349599155.jpeg', '4510360143214.jpeg', '4510391496337.jpeg',
       '4510470343722.jpeg', '4510477439417.jpeg', '4510493977198.jpeg',
       '4510502490698.jpeg', '4510512676102.jpeg', '4510548785812.jpeg',
       '4510557642488.jpeg', '4510584940796.jpeg', '4510586850002.jpeg',
       '4510587596995.jpeg', '4510625182719.jpeg', '4510652970126.jpeg',
       '4510665840215.jpeg', '4510667048453.jpeg', '4510670759491.jpeg',
       '4510697098042.jpeg', '4510700555591.jpeg', '4510703187368.jpeg',
       '4510706473457.jpeg', '4510716972216.jpeg', 

In [404]:
# # 学習データとテストデータのインデックスを作成
# train_index, test_index = next(ss.split(file_names))

In [405]:
# file_names_train, file_names_test = file_names[train_index], file_names[test_index] # 学習データ，テストデータ

### 画像をnumpy配列にするための関数

In [406]:
def jpeg_to_numpy(image_path):
    # JPEG画像を開く
    image = Image.open(image_path)
    # NumPy配列に変換
    numpy_array = np.array(image)
    
    return numpy_array

def convert_to_grayscale(numpy_array):
    # グレーと言わず2値化
    grayscale_array = np.where(numpy_array <= 128, 0, 255)
    # plt.imshow(grayscale_array) # こいつらのせいで処理が重かった。出力系は要注意
    # print(grayscale_array)
    return grayscale_array


### 画像を配列にしてよしなに

In [407]:
X, y = [], []

In [408]:
for file_name in file_names:
    numpy_array = jpeg_to_numpy(DIRS_DATASET + file_name)
    grayscale_array = convert_to_grayscale(numpy_array)
    for i in range(13):
        X.append(np.where(grayscale_array == 255, i+1, 0))
        y.append(file_name[i])

In [409]:
X = np.array(X)
y = np.array(y, dtype=int)

In [410]:
X

array([[[[ 0,  0,  0],
         [ 1,  1,  1],
         [ 0,  0,  0],
         ...,
         [ 1,  1,  1],
         [ 1,  1,  1],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 2,  2,  2],
         [ 0,  0,  0],
         ...,
         [ 2,  2,  2],
         [ 2,  2,  2],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 3,  3,  3],
         [ 0,  0,  0],
         ...,
         [ 3,  3,  3],
         [ 3,  3,  3],
         [ 0,  0,  0]]],


       ...,


       [[[ 0,  0,  0],
         [11, 11, 11],
         [ 0,  0,  0],
         ...,
         [11, 11, 11],
         [11, 11, 11],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [12, 12, 12],
         [ 0,  0,  0],
         ...,
         [12, 12, 12],
         [12, 12, 12],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [13, 13, 13],
         [ 0,  0,  0],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [ 0,  0,  0]]]])

In [411]:
y

array([4, 5, 1, 0, 0, 3, 3, 3, 9, 7, 5, 2, 8, 4, 5, 1, 0, 0, 5, 0, 6, 3,
       4, 6, 9, 9, 4, 5, 1, 0, 0, 7, 2, 9, 4, 8, 9, 4, 4, 4, 5, 1, 0, 0,
       7, 7, 3, 5, 1, 0, 7, 6, 4, 5, 1, 0, 1, 0, 4, 2, 3, 7, 6, 1, 2, 4,
       5, 1, 0, 1, 1, 6, 1, 2, 9, 8, 2, 6, 4, 5, 1, 0, 1, 2, 5, 9, 5, 0,
       2, 2, 2, 4, 5, 1, 0, 1, 3, 5, 9, 1, 6, 1, 5, 2, 4, 5, 1, 0, 1, 4,
       8, 1, 0, 1, 3, 7, 9, 4, 5, 1, 0, 1, 5, 4, 8, 3, 7, 8, 4, 4, 4, 5,
       1, 0, 1, 5, 5, 4, 6, 2, 1, 7, 8, 4, 5, 1, 0, 1, 8, 9, 3, 8, 9, 1,
       3, 2, 4, 5, 1, 0, 1, 9, 2, 3, 4, 0, 5, 4, 6, 4, 5, 1, 0, 2, 4, 5,
       1, 4, 8, 3, 6, 4, 4, 5, 1, 0, 2, 5, 0, 1, 6, 8, 8, 7, 8, 4, 5, 1,
       0, 2, 8, 3, 7, 2, 4, 2, 9, 9, 4, 5, 1, 0, 2, 9, 9, 1, 5, 2, 0, 4,
       0, 4, 5, 1, 0, 3, 1, 1, 6, 5, 3, 2, 5, 5, 4, 5, 1, 0, 3, 4, 9, 5,
       9, 9, 1, 5, 5, 4, 5, 1, 0, 3, 6, 0, 1, 4, 3, 2, 1, 4, 4, 5, 1, 0,
       3, 9, 1, 4, 9, 6, 3, 3, 7, 4, 5, 1, 0, 4, 7, 0, 3, 4, 3, 7, 2, 2,
       4, 5, 1, 0, 4, 7, 7, 4, 3, 9, 4, 1, 7, 4, 5,

In [412]:
# ラベルデータをone-hotベクトルに直す
labels = {
    0: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    1: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 
    2: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 
    3: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 
    4: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 
    5: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 
    6: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 
    7: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 
    8: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 
    9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 
}

y = np.array(list(map(lambda v : labels[v] , y)))

In [413]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

### データの分割

In [414]:
ss = ShuffleSplit(n_splits=1,      # 分割を1個生成
                  train_size=0.8,  # 学習
                  test_size =0.2,  # テスト
                  random_state=0)  # 乱数種（再現用）

# 学習データとテストデータのインデックスを作成
train_index, test_index = next(ss.split(X))

X_train, X_test = X[train_index], X[test_index] # 学習データ，テストデータ
y_train, y_test = y[train_index], y[test_index] # 学習データのラベル，テストデータのラベル

In [415]:
X

array([[[[ 0,  0,  0],
         [ 1,  1,  1],
         [ 0,  0,  0],
         ...,
         [ 1,  1,  1],
         [ 1,  1,  1],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 2,  2,  2],
         [ 0,  0,  0],
         ...,
         [ 2,  2,  2],
         [ 2,  2,  2],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 3,  3,  3],
         [ 0,  0,  0],
         ...,
         [ 3,  3,  3],
         [ 3,  3,  3],
         [ 0,  0,  0]]],


       ...,


       [[[ 0,  0,  0],
         [11, 11, 11],
         [ 0,  0,  0],
         ...,
         [11, 11, 11],
         [11, 11, 11],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [12, 12, 12],
         [ 0,  0,  0],
         ...,
         [12, 12, 12],
         [12, 12, 12],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [13, 13, 13],
         [ 0,  0,  0],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [ 0,  0,  0]]]])

In [416]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [417]:
X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3] 

(104000, 1, 112, 3)

In [418]:
X_train

array([[[[ 0,  0,  0],
         [ 3,  3,  3],
         [ 0,  0,  0],
         ...,
         [ 3,  3,  3],
         [ 3,  3,  3],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 1,  1,  1],
         [ 0,  0,  0],
         ...,
         [ 1,  1,  1],
         [ 1,  1,  1],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 2,  2,  2],
         [ 0,  0,  0],
         ...,
         [ 2,  2,  2],
         [ 2,  2,  2],
         [ 0,  0,  0]]],


       ...,


       [[[ 0,  0,  0],
         [13, 13, 13],
         [ 0,  0,  0],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 5,  5,  5],
         [ 0,  0,  0],
         ...,
         [ 5,  5,  5],
         [ 5,  5,  5],
         [ 0,  0,  0]]],


       [[[ 0,  0,  0],
         [ 6,  6,  6],
         [ 0,  0,  0],
         ...,
         [ 6,  6,  6],
         [ 6,  6,  6],
         [ 0,  0,  0]]]])

### ニューラルネットワークの構築

In [419]:

# 学習し、テストデータで評価し、スコアを表示する
# 引数は、中間層の数、バッチサイズ、epoch数

def fit_epoch(neuron, batch, epochs):
    
    # レイヤーのオブジェクトを作成
    Dense = keras.layers.Dense

    # モデルの構造を定義
    model = keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(1, 112, 3)))
    model.add(Dense(neuron, activation='relu'))
    
    # 畳み込み層を追加
    # model.add(tf.keras.layers.Flatten(tf.keras.layers.Conv2D(filters=neuron, kernel_size=(3, 3), activation='relu', input_shape=(1, 112, 3))))

    model.add(Dense(10, activation='softmax')) # 10つのラベルがありsoftmaxで最後の層作る

    # モデルを構築
    model.compile(
    loss='categorical_crossentropy',
    optimizer='Adamax',
    metrics=['accuracy'])

    # 学習を実行
    hist = model.fit(X_train, y_train,
        batch_size=batch, # 誤差逆伝播法をするときの1回当たりのデータ数
        epochs=epochs,
        verbose=1,
        validation_split=0.1)
    
   # モデルを評価
    score = model.evaluate(X_test, y_test, verbose=1)
    print('正解率=', score[1], 'loss=', score[0])
    
     # 学習の様子をグラフへ描画 
    # 正解率の推移をプロット
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('Accuracy')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # ロスの推移をプロット
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Loss')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()   

In [422]:
%%time
fit_epoch(64, 1, 2000)

Epoch 1/2000
93600/93600 [==============================] - 116s 1ms/step - loss: 2.2413 - accuracy: 0.1785 - val_loss: 2.1927 - val_accuracy: 0.1833
Epoch 2/2000
93600/93600 [==============================] - 118s 1ms/step - loss: 2.1805 - accuracy: 0.1846 - val_loss: 2.1946 - val_accuracy: 0.1874
Epoch 3/2000
93600/93600 [==============================] - 122s 1ms/step - loss: 2.1794 - accuracy: 0.1831 - val_loss: 2.1876 - val_accuracy: 0.1855
Epoch 4/2000
93600/93600 [==============================] - 124s 1ms/step - loss: 2.1793 - accuracy: 0.1839 - val_loss: 2.1905 - val_accuracy: 0.1826
Epoch 5/2000
16901/93600 [====>.........................] - ETA: 1:36 - loss: 2.1858 - accuracy: 0.1814

KeyboardInterrupt: 